<a href="https://colab.research.google.com/github/Sadiya-Afroz/23455777-Sadiya-Afroz/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [2]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install pyinputplus
!pip install matplotlib
!pip install requests
!pip install ollama

  Using cached ollama-0.6.0-py3-none-any.whl.metadata (4.3 kB)
Using cached ollama-0.6.0-py3-none-any.whl (14 kB)


In [ ]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au/'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

## 📦 Setup and Configuration
Import required packages and setup environment.

In [4]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather

import json
import re

In [6]:
from ollama import chat
from ollama import ChatResponse
from ollama import Client

## 🌤️ Weather Data Functions

In [ ]:
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location using the wttr.in API.

    Args:
        location (str): City or location name (e.g. "Perth")
        forecast_days (int): Number of days to forecast (1–5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    try:
        # Build the API URL. wttr.in returns JSON when format=j1
        url = f"https://wttr.in/{location}?format=j1"

        # Send a GET request to the API
        response = requests.get(url)
        response.raise_for_status()  # Raises error if response is bad

        # Convert the API response to a Python dictionary
        weather = response.json()

        if not weather:
            print("⚠️ No data returned. Check location spelling or internet connection.")
            return None

        # Only keep requested number of forecast days (1–5)
        if "weather" in weather:
            weather["weather"] = weather["weather"][:forecast_days]

        return weather

    except requests.exceptions.RequestException as e:
        print("❌ Network or connection error:", e)
        return None
    except Exception as e:
        print("❌ Error fetching weather data:", e)
        return None


In [38]:
print(get_weather_data("Perth"))

{'current_condition': [{'FeelsLikeC': '10', 'FeelsLikeF': '49', 'cloudcover': '25', 'humidity': '87', 'localObsDateTime': '2025-10-19 09:29 PM', 'observation_time': '01:29 PM', 'precipInches': '0.0', 'precipMM': '0.0', 'pressure': '1011', 'pressureInches': '30', 'temp_C': '11', 'temp_F': '52', 'uvIndex': '0', 'visibility': '10', 'visibilityMiles': '6', 'weatherCode': '176', 'weatherDesc': [{'value': 'Patchy rain nearby'}], 'weatherIconUrl': [{'value': ''}], 'winddir16Point': 'SW', 'winddirDegree': '230', 'windspeedKmph': '11', 'windspeedMiles': '7'}], 'nearest_area': [{'areaName': [{'value': 'Maylands'}], 'country': [{'value': 'Australia'}], 'latitude': '-31.933', 'longitude': '115.883', 'population': '10447', 'region': [{'value': 'Western Australia'}], 'weatherUrl': [{'value': ''}]}], 'request': [{'query': 'Lat -31.95 and Lon 115.86', 'type': 'LatLon'}], 'weather': [{'astronomy': [{'moon_illumination': '6', 'moon_phase': 'Waning Crescent', 'moonrise': '04:13 AM', 'moonset': '04:35 PM'

## 📊 Visualisation Functions

In [ ]:
import matplotlib.pyplot as plt

def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create a line chart showing the average temperature for each forecast day.

    Args:
        weather_data (dict): The weather data returned by get_weather_data()
        output_type (str): 'display' to show in notebook, 'figure' to return the chart object

    Returns:
        If output_type == 'figure', returns the matplotlib Figure object.
        Otherwise, displays the chart directly in the notebook.
    """
    # Try to get forecast section
    days = weather_data.get("weather", [])
    if not days:
        print("⚠️ No forecast data available.")
        return

    dates = []
    temps = []

    for day in days:
        date = day.get("date", "N/A")
        avg_temp = day.get("avgtempC") or day.get("avgtemp", 0)
        try:
            avg_temp = int(avg_temp)
        except Exception:
            avg_temp = 0
        dates.append(date)
        temps.append(avg_temp)

    if not any(temps):
        print("Temperature data missing or zero — check your data source.")
        return

    fig = plt.figure()
    plt.plot(dates, temps, marker='o', color='orange', linewidth=2)
    plt.title("Average Temperature Over Time (°C)")
    plt.xlabel("Date")
    plt.ylabel("Temperature (°C)")
    plt.grid(True)

    if output_type == 'figure':
        return fig
    else:
        plt.show()



def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create a bar chart showing the chance of rain for each forecast day.

    Args:
        weather_data (dict): The weather data returned by get_weather_data()
        output_type (str): 'display' to show in notebook, 'figure' to return the chart object

    Returns:
        If output_type == 'figure', returns the matplotlib Figure object.
        Otherwise, displays the chart directly in the notebook.
    """
    days = weather_data.get("weather", [])
    if not days:
        print("⚠️ No forecast data available.")
        return

    dates = []
    rain_chances = []

    for day in days:
        date = day.get("date", "N/A")

        # Safely access hourly data
        hourly = day.get("hourly", [])
        if not hourly:
            rain_chance = 0
        else:
            # Choose midday record if available
            midday = hourly[4] if len(hourly) > 4 else hourly[0]
            # Some APIs use 'chanceofrain', others use 'chance_of_rain'
            rain_chance = midday.get("chanceofrain") or midday.get("chance_of_rain") or 0
            try:
                rain_chance = int(rain_chance)
            except Exception:
                rain_chance = 0

        dates.append(date)
        rain_chances.append(rain_chance)

    if not any(rain_chances):
        print("Warning: No precipitation data found — check your data structure or API mode.")
        return

    fig = plt.figure()
    plt.bar(dates, rain_chances, color='skyblue')
    plt.title("Chance of Rain (%)")
    plt.xlabel("Date")
    plt.ylabel("Chance (%)")
    plt.ylim(0, 100)

    if output_type == 'figure':
        return fig
    else:
        plt.show()


## 🤖 Natural Language Processing

In [ ]:
import os

def get_response(prompt):
    """
    Get a response from ollama server chat model.

    Args:
        prompt (str): The input prompt to send to the model.

    Returns:
        str: The model's response.
    """
    try:
        client = Client( host="https://ollama.com",
            headers={
                'Authorization': 'Bearer ' + os.environ.get('OLLAMA_API_KEY')
            }
        )

        response = client.chat(model='gpt-oss:120b', messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ])
    except Exception as e:
        print("Error getting response from model:", e)
        return ""
    
    return response['message']['content']

# print(get_response("Hello, how are you?"))


Hello! I'm doing great, thank you for asking. How can I assist you today?


In [ ]:
def safe_json_loads(json_string):
    """
    Safely load a JSON string, returning an empty dict on failure.

    Args:
        json_string (str): The JSON string to parse.
    Returns:
        dict: The parsed JSON object, or empty dict on failure.
    """
    
    match = re.search(r'\{.*\}', json_string, re.DOTALL)
    if match:
        json_string = match.group(0)
    try:
        return json.loads(json_string)
    except json.JSONDecodeError:
        print("Error: Failed to parse JSON from model response.")
        return {}
    

safe_json_loads('``{"location": "Sydney", "time_period": "tomorrow", "weather_attribute": "rain"}')

{'location': 'Sydney', 'time_period': 'tomorrow', 'weather_attribute': 'rain'}

In [ ]:
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question (e.g., "Will it rain tomorrow in Sydney?")

    Returns:
        dict: Extracted information including location, time period, and weather attribute.
              Example: {"topic": "rain", "location": "Sydney", "time": "tomorrow"}
    """
    # Make everything lowercase for easier comparison
    
    prompt = f"""
    Extract location, time period, and weather attribute from
    the following question:  {question}
    CRITICAL: Return the result as a JSON
    object with keys
    - 'location': the specified location, for example Perth, London, Paris (string, Perth is none)
    - 'time_period': the time period for example, 'today', 'tomorrow', 'next week' (string, today is none)
    - 'weather_attribute': the weather attribute for example, 'temperature', 'precipitation' (string, general is none)
    """

    # Return all extracted info
    reponse = get_response(prompt)
    json = safe_json_loads(reponse)

    return json


print(parse_weather_question("Will it rain tomorrow in Sydney?"))

## 🧭 User Interface

In [9]:
def show_current_weather_data():
    pass


In [ ]:
def run_all_at_once():
    """
    Run one full weather check session:
    - Ask for a city
    - Display current summary
    - Show temperature and rain charts
    - Answer one natural language question
    """
    
    # Ask for a city name
    city = pyip.inputStr("Enter a city name (e.g., Perth, Sydney): ")

    # Fetch weather data
    weather_data = get_weather_data(city)
    if not weather_data:
        print("⚠️ Could not fetch data. Please try again.")
        return

    # Display current summary
    print("\n📋 Current Weather Summary:")
    print(generate_weather_response({"topic": "general"}, weather_data))

    # Show charts
    print("\n📈 Generating charts...")
    create_temperature_visualisation(weather_data)
    create_precipitation_visualisation(weather_data)

    # Let user ask a question
    question = pyip.inputStr("\n💬 Ask a weather question (e.g., Will it rain tomorrow in Perth?): ")
    parsed = parse_weather_question(question)

    # Use the location from the question if provided
    if parsed.get("location") and parsed["location"].lower() != city.lower():
        weather_data = get_weather_data(parsed["location"])

    # Generate and show the response
    print("\n🤖 Answer:")
    print(generate_weather_response(parsed, weather_data))

run_all_at_once()

In [ ]:

def main_menu():
    """
    A simple text-based main menu using PyInputPlus.
    """

    print("🌦️ Welcome to WeatherWise!")

    while True:
        choice = pyip.inputMenu(
            ["Current weather data", "Ask about weather", "Visualizations", "Run all together" ,"Quit"],
            numbered=True,
            prompt="\nMain Menu - Choose an option:\n"
        )

        if choice == "Current weather data":
            show_current_weather_data()

        elif choice == "Ask about weather":
            question = pyip.inputStr("\nAsk a weather question (e.g., Will it rain tomorrow in Perth?): ")
            parsed = parse_weather_question(question)
            weather_data = get_weather_data(parsed.get("location", city))
            print("\n🤖 Answer:")
            # print(generate_weather_response(parsed, weather_data))

        elif choice == "Visualizations":
            city = pyip.inputStr("Enter a city name (e.g., Perth, Sydney): ")

            v_type = pyip.inputMenu(
                ["Temperature Chart", "Precipitation Chart", "Both Charts"],
                numbered=True,
                prompt="\nChoose a visualization type:\n"
            )

            # weather_data = get_weather_data(city)
            # create_temperature_visualisation(weather_data)
            # create_precipitation_visualisation(weather_data)

        elif choice == "Run all together":
            run_all_at_once()
            
        elif choice == "Quit":
            print("👋 Goodbye! Stay safe and check the weather again soon.")
            break

        else:
            print("NOT A VALID OPTION")

main_menu()


🌦️ Welcome to WeatherWise!

Main Menu - Choose an option:
1. Current weather data
2. Ask about weather
3. Visualizations
4. Quit
👋 Goodbye! Stay safe and check the weather again soon.


## 🧩 Main Application Logic

In [40]:
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data (from parse_weather_question)
        weather_data (dict): Weather data (from get_weather_data)

    Returns:
        str: Human-readable weather answer
    """
    if not weather_data:
        return "⚠️ Sorry, I couldn’t find any weather information right now."

    try:
        # Extract basic current weather details
        current = weather_data.get("current_condition", [{}])[0]
        temp_c = current.get("temp_C", "N/A")
        condition = current.get("weatherDesc", [{"value": "Unknown"}])[0]["value"]
        humidity = current.get("humidity", "N/A")
        wind = current.get("windspeedKmph", "N/A")

        topic = parsed_question.get("topic", "general")
        time = parsed_question.get("time", "today")

        # 🌧️ RAIN
        if topic == "rain":
            today = weather_data.get("weather", [{}])[0]
            hourly = today.get("hourly", [])
            midday = hourly[4] if len(hourly) > 4 else (hourly[0] if hourly else {})
            chance = midday.get("chanceofrain") or midday.get("chance_of_rain") or "N/A"
            response = (
                f"☔ The chance of rain {time} is around {chance}%. "
                f"Current condition: {condition.lower()}."
            )

        # 🌡️ TEMPERATURE
        elif topic == "temperature":
            response = (
                f"🌡️ The temperature {time} is about {temp_c}°C "
                f"with {condition.lower()} conditions."
            )

        # 🌬️ WIND
        elif topic == "wind":
            response = (
                f"🌬️ The wind speed {time} is about {wind} km/h. "
                f"It’s currently {condition.lower()}."
            )

        # 💧 HUMIDITY
        elif topic == "humidity":
            response = (
                f"💧 The humidity {time} is about {humidity}%. "
                f"It’s {condition.lower()} outside."
            )

        # 🌤️ GENERAL SUMMARY
        else:
            response = (
                f"🌤️ Right now it’s {temp_c}°C with {condition.lower()}, "
                f"humidity {humidity}%, and wind speed around {wind} km/h."
            )

        return response

    except Exception as e:
        return f"⚠️ Sorry, I couldn’t generate the weather response. ({e})"


## 🧪 Testing and Examples

In [12]:
# Include sample input/output for each function
from hands_on_ai.chat import get_response

def hands_on_ai_example():
    """
    Example of using Hands-On AI to generate a weather response.
    """
    prompt = (
        "The current weather in Perth is 25°C with light rain. "
        "Humidity is 60% and wind speed is 15 km/h. "
        "Will it rain tomorrow?"
    )

    response = get_response(prompt)
    print("🤖 Hands-On AI Response:")
    print(response)
print(hands_on_ai_example())

[WARNING] Error during request (attempt 1): 405 method not allowed


Hang tight, I'm thinking... trying again!


[WARNING] Error during request (attempt 2): 405 method not allowed


🤖 Hands-On AI Response:
❌ Error: 405 method not allowed
None


## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.